This notebook contains example visualizations from the CM1-SDM grid-aggregated outputs.

In [2]:
# Using the NPL 2024a kernel
import numpy as np
import xarray as xr
import os.path
import os
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.ticker
from matplotlib import animation
from IPython.display import HTML
%matplotlib inline

# Example 1: Compare two cm1out files

Two files of interest:  
path1 = '/glade/campaign/univ/upsu0052/kkchandr/rhod_runs/outsdm_iceball_nowind_rhod_dist_min200_sgs_1024_poly/cm1out.nc'
path2 = '/glade/derecho/scratch/klamb/superdroplets/outsdm_iceball_nowind_rhod_dist_min200_sgs_1024_poly_trj_5400_7200/cm1out_only_upto35.nc'

In [3]:
path1 = '/glade/campaign/univ/upsu0052/kkchandr/rhod_runs/outsdm_iceball_nowind_rhod_dist_min200_sgs_1024_poly/cm1out.nc'
path2 = '/glade/derecho/scratch/klamb/superdroplets/outsdm_iceball_nowind_rhod_dist_min200_sgs_1024_poly_trj_5400_7200/cm1out_only_upto35.nc'
ds1 = xr.open_dataset(path1)
ds2 = xr.open_dataset(path2)
# get the intersection of two datasets, on time dimension
ds2 = ds2.drop_duplicates(dim='time', keep='first') # drop duplicate time steps, keep the first instance
time_idx = np.isin(ds1.time, ds2.time) # get indices where timestamps of ds1 are in timestamps of ds2
ds1 = ds1.sel(time=time_idx, drop=True) # get subset based on timestamps
ds2 = ds2.sortby('time') # sort by timestamps

In [4]:
# set up a dictionary to hold the two datasets
ds_dict = {}
vars = ['qi', 'xh', 'yh', 'z'] # define variables of interest
# average over the y-axis (i.e., keep only x- and z- axes)
ds_dict['ds1'] = ds1[vars].mean(dim='nj')
ds_dict['ds2'] = ds2[vars].mean(dim='nj')

Initialize the plot  
Note: %%capture used here prevents the initiallized plot from outputting

In [38]:
%%capture
var = 'qi' # set variable of interest
# find absolute min/max across all experiments
x_min = 0
x_max = 0 
for key in ds_dict.keys():
    ds = ds_dict[key]
    x_max_local = ds[var].max().item()
    x_min_local = ds[var].min().item()
    if x_max_local > x_max:
        x_max = x_max_local
    if x_min_local < x_min:
        x_min = x_min_local
print(x_min, x_max)

fig, ax_grid = plt.subplots(nrows=1, 
                            ncols=2, 
                            figsize=(8, 3)) # set up 1x2 grid for plot
var_list = ds_dict.keys()
cbformat = matplotlib.ticker.ScalarFormatter()   # create the formatter
cbformat.set_powerlimits((-2,2)) # set limits for when scientific notation representation is used
counter = 0 
im_list = [] # list to hold plot obj 
for i in range(2):
    ax = ax_grid[i]
    exp_name = list(var_list)[counter]
    ds = ds_dict[exp_name]
    da = ds[var]
    im = ax.pcolormesh(ds.xh, ds.z, da[35, :, :], cmap='Greys_r', alpha=0.8)
    im_list.append(im)
    im.set_clim(x_min, 0.9*x_max)
    ax.set_xlabel('x [km]', fontsize=10)
    ax.set_ylabel('z [km]', fontsize=10)
    cbar = plt.colorbar(im, ax=ax, fraction=0.05, pad=0.05, format=cbformat)
    cbar.set_label('mixing ratio [kg/kg]')
    counter += 1
fig.tight_layout()
fig.subplots_adjust(wspace=0.5)

In [39]:
# create function that will be called by animation object
frames = 70 # define number of time steps in animation
def animate(i):
    counter = 0
    for exp_name in list(var_list):
        ds = ds_dict[exp_name]
        da = ds[var]
        array = da[i, :, :].values
        im_list[counter].set_array(array.flatten())
        seconds = da.time[i].values / np.timedelta64(1, 's')
        ax_grid[counter].set_title(f'{exp_name}, {str(int(seconds))} seconds', fontsize=8)
        counter += 1

In [40]:
# create animation object
save_dir = '/glade/u/home/joko'
ani = animation.FuncAnimation(fig, animate, frames, interval=150)
HTML(ani.to_jshtml())
# ani.save(f'{save_dir}/iceball_bulk_les_grid.gif') # save as gif